### single node triton server with load test
1. Loading to and from minio workflow (multiple models)
2. Getting models from [timm](https://timm.fast.ai/)
3. Making the node
4. Load and unload models operations
5. Monitoring
6. Naive load test
7. TODO Add language models from Huggingface


In [18]:
import timm
import torch
from PIL import Image
import os
import numpy as np
import pandas as pd

In [19]:
len(timm.list_models())

739

In [20]:
os.system('sudo umount -l ~/my_mounting_point')
os.system('cc-cloudfuse mount ~/my_mounting_point')

data_folder_path = '/home/cc/my_mounting_point/datasets'
dataset_folder_path = os.path.join(
    data_folder_path, 'ILSVRC/Data/DET/test'
)
classes_file_path = os.path.join(
    data_folder_path, 'imagenet_classes.txt'
)

image_names = os.listdir(dataset_folder_path)
image_names.sort()
with open(classes_file_path) as f:
    classes = [line.strip() for line in f.readlines()]

def image_loader(folder_path, image_name):
    image = Image.open(
        os.path.join(folder_path, image_name))
    # if there was a need to filter out only color images
    # if image.mode == 'RGB':
    #     pass
    return image
num_loaded_images = 20
images = {
    image_name: image_loader(
        dataset_folder_path, image_name) for image_name in image_names[
            :num_loaded_images]}

In [21]:
# load and transform model
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)])

batch = torch.stack(list(map(lambda a: transform(a), list(images.values()))))

In [22]:
# load and predict with the timm model
model_name = 'resnet50'
model = timm.create_model(model_name, pretrained=True)

model.eval()
torch_output = model(batch)
torch_output = torch.nn.functional.softmax(torch_output, dim=1) * 100
torch_output = torch_output.detach().numpy()
torch_output = torch_output.argmax(axis=1)
torch_class = np.array(classes)[torch_output]
torch_class

array(['lion', 'studio_couch', 'harp', 'goldfish', 'snowplow',
       'pomegranate', 'alligator_lizard', 'stethoscope', 'banjo', 'junco',
       'shoe_shop', 'albatross', 'notebook', 'lesser_panda', 'microphone',
       'football_helmet', 'damselfly', 'barrel', 'harp', 'mortarboard'],
      dtype='<U30')

In [24]:
# save the onnx model
import torch.onnx

if 'models' not in os.listdir("./"):
    os.mkdir('models')
# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.
dummy_input = torch.randn(1, 3, 224, 224)
# Invoke export
torch.onnx.export(
    model, dummy_input,
    f"./models/{model_name}.onnx",
    input_names = ['input'],
    output_names = ['output'],
    dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                  'output' : {0 : 'batch_size'}})

In [25]:
# use onnx model
import onnx
import onnxruntime

onnx_model = onnx.load(f"./models/{model_name}.onnx")
onnx.checker.check_model(onnx_model)

ort_session = onnxruntime.InferenceSession(f"{model_name}.onnx", providers=['CPUExecutionProvider'])
onnx_output = ort_session.run(None, {'input': batch.numpy()})
onnx_output = torch.nn.functional.softmax(torch.tensor(onnx_output), dim=1)[0] * 100
onnx_output = onnx_output.detach().numpy()
onnx_output = onnx_output.argmax(axis=1)
onnx_class = np.array(classes)[onnx_output]
onnx_class

/tmp/ipykernel_4050229/3273098817.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  onnx_output = torch.nn.functional.softmax(torch.tensor(onnx_output), dim=1)[0] * 100


array(['lion', 'studio_couch', 'slot', 'goldfish', 'snowplow',
       'pomegranate', 'alligator_lizard', 'seat_belt', 'hatchet', 'junco',
       'shoe_shop', 'albatross', 'bookshop', 'lesser_panda', 'marimba',
       'football_helmet', 'damselfly', 'horse_cart', 'harp',
       'mortarboard'], dtype='<U30')

In [26]:
# TODO find out why slightly different

assert np.all(onnx_output == torch_output)
print(onnx_class)

AssertionError: 

### Deploying on Onnx Docker

In [12]:
VERSION='22.05'
os.system(f"docker pull nvcr.io/nvidia/tritonserver:{VERSION}-py3")
# add --gpus=<number of gpus> on gpu machines
os.system("docker run --rm -p8000:8000 -p8001:8001 -p8002:8002"
          f" -v {os.getcwd()}/models:/models "
          f"nvcr.io/nvidia/tritonserver:{VERSION}-py3 tritonserver --model-repository=/models")
# print("docker run --rm -p8000:8000 -p8001:8001 -p8002:8002"
#           f" -v {os.getcwd()}/models:/models "
#           f"nvcr.io/nvidia/tritonserver:{VERSION}-py3 tritonserver --model-repository=/models")

22.05-py3: Pulling from nvidia/tritonserver
Digest: sha256:a85daa2907f46e70b3782818a0331df62d9b4e0b1f15f1530b2a52c8c782d46d
Status: Image is up to date for nvcr.io/nvidia/tritonserver:22.05-py3
nvcr.io/nvidia/tritonserver:22.05-py3

== Triton Inference Server ==

NVIDIA Release 22.05 (build 38317651)
Triton Server Version 2.22.0

Copyright (c) 2018-2022, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION & AFFILIATES.  All rights reserved.

This container image and its contents are governed by the NVIDIA Deep Learning Container License.
By pulling and using the container, you accept the terms and conditions of this license:
https://developer.nvidia.com/ngc/nvidia-deep-learning-container-license

   Use the NVIDIA Container Toolkit to start this container with GPU support; see
   https://docs.nvidia.com/datacenter/cloud-native/ .



W0624 18:56:30.001279 1 pinned_memory_manager.cc:236] Unable to allocate pinned system memory, pinned memory pool will not be available: CUDA driver version is insufficient for CUDA runtime version
I0624 18:56:30.001530 1 cuda_memory_manager.cc:115] CUDA memory pool disabled
E0624 18:56:30.002653 1 model_repository_manager.cc:2063] Poll failed for model directory 'resnet': model output has different size for dims and reshape for resnet
I0624 18:56:30.002702 1 server.cc:556] 
+------------------+------+
| Repository Agent | Path |
+------------------+------+
+------------------+------+

I0624 18:56:30.002720 1 server.cc:583] 
+---------+------+--------+
| Backend | Path | Config |
+---------+------+--------+
+---------+------+--------+

I0624 18:56:30.002737 1 server.cc:626] 
+-------+---------+--------+
| Model | Version | Status |
+-------+---------+--------+
+-------+---------+--------+

I0624 18:56:30.002948 1 tritonserver.cc:2138] 
+----------------------------------+--------------

256

In [3]:
# TODO use python to sent requests

### Multi Model

In [29]:
# Load a bunch of models

### Model Switching

### Model load and offload

In [13]:
onnx_model

NameError: name 'onnx_model' is not defined

### Seldon

In [4]:
# add model switching etc